# Ejercicio 1 - Suma filas y columnas de matriz

Partiendo del código suministrado:

* Completar la función que suma las columnas de la matriz

* Añadir la invocación del nuevo `kernel col_sum` y la comprobación de resultados

* Comparar las prestaciones de la suma por filas y por columnas

---

In [ ]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[idx][i]
    sums[idx] = sum

# TODO: completar
@cuda.jit
def col_sums(a, sums, n):
    None

n = 32768 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

# Input Matrix
h_a = np.ones(n*n).reshape(n, n).astype(np.float32) 

# Vectors in GPU
d_a = cuda.to_device(h_a)
d_sums = cuda.device_array(shape=(n,), dtype=np.float32)

# Calcular suma de filas
row_sums[blocks, threads_per_block](d_a, d_sums, n)
h_sums = d_sums.copy_to_host()
# Comprobar suma
truth = h_a.sum(axis=1)
np.testing.assert_equal(h_sums,truth)

# TODO: calcular y comprobar suma de columnas

/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/dispatcher.py:605: NumbaPerformanceWarning: Grid size 64 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))



Tiempo en CPU:
3.83 µs ± 867 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Tiempo en GPU:
51.1 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Sergio

In [ ]:
import numpy as np
from numba import cuda

@cuda.jit
def row_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[idx][i]
    sums[idx] = sum

# TODO: completar
@cuda.jit
def col_sums(a, sums, n):
    idx = cuda.grid(1)
    sum = 0.0
    for i in range(n):
        sum += a[i][idx]
    sums[idx] = sum

n = 32768 # matrix side size
threads_per_block = 256
blocks = int(n / threads_per_block)

# Input Matrix

h_a = np.random.randint(low=1, high= 100, size=n*n).reshape(n, n).astype(np.float32) 

# Vectors in GPU
d_a = cuda.to_device(h_a)
d_sums = cuda.device_array(shape=(n,), dtype=np.float32)

# Calcular suma de filas
row_sums[blocks, threads_per_block](d_a, d_sums, n)
h_sums = d_sums.copy_to_host()
# Comprobar suma
truth = h_a.sum(axis=1)
np.testing.assert_equal(h_sums,truth)

# TODO: calcular y comprobar suma de columnas
d_sums = cuda.device_array(shape=(n,), dtype=np.float32)
col_sums[blocks, threads_per_block](d_a, d_sums, n)
h_sums = d_sums.copy_to_host()
# Comprobar suma
truth = h_a.sum(axis=0)
np.testing.assert_equal(h_sums,truth)

In [ ]:
%timeit row_sums[blocks, threads_per_block](d_a, d_sums, n); cuda.synchronize()
%timeit col_sums[blocks, threads_per_block](d_a , d_sums, n); cuda.synchronize()